In [7]:
import numpy as np

In [8]:
class Sequential:
    def __init__(self, layers=None):
        self.layers = layers if layers is not None else []
        self.loss = None
        self.optimiser = None

    def add(self, layer):
        self.layers.append(layer)

    def forward_prop(self, inputs):
        for layer in self.layers:
            inputs = layer.forward_prop(inputs)
        return inputs

    def backward_prop(self, dvalues):
        for layer in reversed(self.layers):
            dvalues = layer.backward_prop(dvalues)

            self.optimiser.update_params(layer)

    def compile(self, loss, optimiser):
        self.loss = loss
        self.optimiser = optimiser

    def fit(self, X, y, epochs=1, batch_size=32):
        # Number of samples
        m = X.shape[0]
        history = {}
        
        for epoch in range(epochs):
            # Shuffle the data at the beginning of each epoch
            indices = np.arange(m)
            np.random.shuffle(indices)
            X = X[indices]
            y = y[indices]
            
            epoch_loss = 0  # Initialise the epoch loss
            for start_idx in range(0, m, batch_size):
                end_idx = min(start_idx + batch_size, m)
                batch_X = X[start_idx:end_idx]
                batch_y = y[start_idx:end_idx]
                
                # Forward propagation
                output = self.forward_prop(batch_X)
                
                # Compute the (average) loss
                loss = np.mean(self.loss.loss(batch_y, output))

                # Accumulate the batch loss
                epoch_loss += loss * (end_idx - start_idx)
                
                # Compute the gradient of the loss with respect to the output
                dvalues = self.loss.derivative(batch_y, output)
                
                # Backward propagation
                self.backward_prop(dvalues)
            
            # Compute average loss
            epoch_loss /= m

            # Store loss in history
            if history.get('loss') is None:
                history['loss'] = [epoch_loss]
            else:
                history['loss'].append(epoch_loss)

            # Print the loss
            print(f"Epoch {epoch + 1}/{epochs}, Batches: {round(m / batch_size)}, Loss: {epoch_loss:.4f}")

        return history

    def predict(self, X):
        # Evaluate the model
        return self.forward_prop(X)